# Building the Hamiltonian for large systems
We now can try to describe a concrete example. Before doing this we need to be able to build Hamiltonians  and learn how to diagonalize it 
Fist build a $N\times  N$ Hermitian matrix. (E.g. set N=2)

In [1]:
import numpy as np
import scipy.linalg as LA 

Now you can diagonalize it, and extract eigenvectors and eigenvalues, check that you can get back the original matrix 
$M = U m U^{\dagger}$

and that 
$M\vec{u}_1 = m_1 \vec{u}_1$


Now we can study our first many body system, the Ising model in transverse field for $N=3$ spins, the Hamiltonian reads
$ H = -\sigma_x^1\otimes \sigma_x^2 - \sigma_x^2\otimes \sigma_x^3 -\sigma_x^3\otimes \sigma_x^1+\lambda \left(  \sigma_z^1+  \sigma_z^2+\sigma_z^3\right)$, build it for several values of $\lambda$ (e.g. 20 points in the interval $[0,10]$) and plot the ground state energy for every value. 

We now move to the generic $N$ we do it recursively notice that going from $N$ to $N+1$ you need to perform a simple step

We wrap the above code into the function by using the 

def function():
return 

statements, we want to provide, $N$ and $\lambda$ as arguments and obtain the energies, the eigenvectors and the Hamiltonian as a result of our function evaluation


Now in the following we use the function just defined to see the first 10 energy levels of the spectrum for several values of the field for fixed $N=10$

Now we can build the translation operator, that cyclically permute all the spins along the chain:
    
it sends spins $1,2 \cdots N-1, N$ into  $N, 1,2 \cdots N-1$

Again we want to create it iteratively starting from $N=2$, notice the simple recursion relation,


Plot the eigenvalues on the complex plane, how do they look like?

Now we build a symmetry operator $S$ which is it? Check that the translation operator, the symmetry operator and the Hamiltonian commute 

Check that the symmetry operators has only eigenvalue $\pm 1$

Now we want to obtain the simultaneous eigenvectors for all three operators  we want to get the spectrum with the three quantum numbers
$|e, k , s\rangle$

$ H|e, k , s\rangle = e |e, k , s\rangle$

$ T|e, k , s\rangle = \exp(ik) |e, k , s\rangle$

$ S|e, k , s\rangle = (-1)^s |e, k , s\rangle$

The trick is to construct a linear combination of the three operators.
You want to build the combination in such a way that the order of the eigenvalues of H is still respected and the other operators are only used to break the degeneracy

e.g for $N=10$ 
$O = H +0.01*T+ 0.001*S$

We only want the first eigenvectors so we will use Lanczos, that is implement in https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.linalg.eigs.html


In [ ]:
from scipy.sparse.linalg import eigs

H,_,_ =construct_and_diagonalize_Hamiltonian(10,1.)
Op_to_diag = H+ 0.01*translation -0.001*symm_op
first_eigenvalue,first_eigenvectors = eigs(Op_to_diag,k=20,which='SR')


Now we have to reconstruct the three eigenvalues we are interested in by using the matrix element of the operator in the desired eigenvealue

$e = \langle e, k , s |H |e, k , s\rangle$

$\exp (i k ) = \langle e, k , s |T |e, k , s\rangle$


$s =  \langle e, k , s |S|e, k , s\rangle$


also since we are interested in $k$ we take the phase of the complex number $\exp(ik)$ using the  cmath.phase() function, remember to import cmath,


Finally we plot all the above information in a dispersion relation, a plot of the Energy  ($e$) versus the momentum ($k$), and use different symbols for different symmetry charges.  



In [ ]:
import cmath

energies =[]
momentum =[]
charges =[]
for k in range(0,20):
    e_v = first_eigenvectors[:,k].T.conj()@H@first_eigenvectors[:,k]
    energies.append(e_v)
    k_v = first_eigenvectors[:,k].T.conj()@translation@first_eigenvectors[:,k]
    momentum.append(cmath.phase(k_v))
    s_v =first_eigenvectors[:,k].T.conj()@symm_op@first_eigenvectors[:,k]
    charges.append(s_v)

print(energies)
print(momentum)
print(charges)
charges =np.array(charges)
energies=np.array(energies)
momentum=np.array(momentum)
print((charges-1))
mask_charges=np.less_equal(np.abs(charges-1),1e-10)

plt.plot(momentum[mask_charges],energies[mask_charges],'.',markersize=20,label='even')
mask_charges=np.less_equal(np.abs(charges+1),1e-10)
plt.plot(momentum[mask_charges],energies[mask_charges],'d',markersize=10,label='odd')
plt.legend(loc="lower right", fontsize=16)
plt.xlabel('Energies', fontsize=16)
plt.ylabel('Momentum', fontsize=16)
plt.grid('on')